In [1]:
# !pip uninstall scispacy 

In [2]:
# !pip install scispacy

In [3]:
# !pip install spacy
# ! pip install en_core_sci_scibert

In [30]:
import scispacy
import spacy
import pandas as pd
import sys
from datetime import datetime 

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import json
from itertools import combinations
from scipy import stats
from collections import Counter

from scispacy.linking import EntityLinker

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [31]:
nlp = spacy.load("en_core_sci_scibert")

In [32]:
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "name": "umls"})
linker = nlp.get_pipe("scispacy_linker")

/users/k1810895/.conda/envs/py39/lib/python3.9/site-packages/spacy/language.py:813: UserWarning: [W119] Overriding pipe name in `config` is not supported. Ignoring override 'umls'.
  warnings.warn(Warnings.W119.format(name_in_config=config.pop("name")))
/users/k1810895/.conda/envs/py39/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/users/k1810895/.conda/envs/py39/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html

In [10]:
# notes = pd.read_csv("/users/k1810895/data/KER/data/mimic_notes.csv")

In [14]:
text = '''This is a 55 y/o M, with H/O Hep C ( presumed from blood transfusions
   s/p 3 rd degree burns in an industrial accident in [**2111**]), with known
   h/o varices who presented on [**3-26**]^th [**2144**] , with abdominal
   distension,jaundice & found to have spontaneous bacterial peritonitis,
   was admitted to [**Hospital Ward Name 383**] 10.
   Change in mentation noted throughout the day, missed his lactulose dose
   [**12-21**] to altered mental status unable to drink safely with risk of
   aspiration, creatinine found to be 8.0 ( Baseline 0.8), BUN in 100
   HD catheter with VIPort inserted & transferred to MICU 6 for further
   management.
   Events :
   Patient dialyzed from [**2165**] hrs to 2230 hrs with only 500 mls fluid
   removal [**12-21**] to Low BP during dialysis.
   IV Albumin 100 grams given as per orders.
   NGT inserted , had minimal bleeding during insertion. (INR 3.7 in am
   lab- had received 4 units of FFP from [**Wardname **]). Had coffee ground NG
   aspirate , connected to low intermittent suction until cleared. Due
   dose of Lactulose given.
   Hepatorenal syndrome
   Assessment:
   Patient is Anuric since am of [**2145-4-1**] as per report received from [**Hospital Ward Name 383**]
   10. Labs drawb prior to Hemodialysis, Please see metavision for lab
   values.
   Action:
   Patient had Hemodialysis done for 2 hours with 500 mls fluid removal
   for the first time.
   Response:
   Renal function continues to be deranged in am labs, continues to be
   anuric.
   Plan:
   Continue monitoring labs, To follow up on next HD
   Electrolyte & fluid disorder, other
   Assessment:
   Action:
   Response:
   Plan:
   Altered mental status (not Delirium)
   Assessment:
   Action:
   Response:
   Plan:
   Hematemesis (upper GI bleed, UGIB)
   Assessment:
   Action:
   Response:
   Plan:
   Bradycardia
   Assessment:
   Action:
   Response:
   Plan:
   Hypotension (not Shock)
   Assessment:
   Action:
   Response:
   Plan:
'''
doc = nlp(text)

# for sent in tokens.sents:
#     print(sent.text, sent.start_char, sent.end_char, sent.start, sent.end)
for sent in doc.sents:
    sentence_tokens.append([token.text for token in sent])


NameError: name 'sentence_tokens' is not defined

#### The formatt {"text": "Directed by Andrei Zagdansky Not rated ; 72 minutes In the documentary '' Orange Winter '' orange blooms throughout Kiev , Ukraine , the epicenter of dissent over that country 's stolen 2004 presidential elections .", "relation": "/location/country/capital", "h": {"id": "m.07t21", "name": "Ukraine", "pos": [123, 130]}, "t": {"id": "m.02sn34", "name": "Kiev", "pos": [116, 120]}}

## Identify concepts from sentences

All text are annoated with UMLS CUIs. Go through each sentence and find CUIs in each sentence. If a pair of CUIs have a relation defined in UMLS, label it as a positive sample.

Cannot go through local json outputs, since json files with the same ids may not contain the same set of documents. Need to load all the data to search, which is too big. 

Medcat does not provide sentence split cutoff in the results. So cannot get sentence infor in annotations. Scispacy provides such information. 

So:
1. Use sentence_seg.sh split documents into sentences. 
2. All annotations have been ingested to ES, which allows query, on openstack. 
3. Go through each sentences resulted from sentence_seg.sh and collect all annotations in the same sentence. 
4. Obtain positive and negative samples from openstack instead of hpc. 


The above process is too complicated. So using NLTK to split sentences and input the annotator to get annotations. 
using *_sentences.sh job scripts. 

Using NLTK is very slow and the outputs from NLTK sentence splitters are different from those from spacy. Finally, rewrite the parser for Medcat output and use the "sentence" object in the scispacy output to produce sentence-level annotations. 


## Enumerate all combinations of entity pairs in sentences

In [38]:
rels = json.load(open('data/rel_map.json', 'r'))

In [39]:
list(rels.items())[:3]

[('C0000039-C0043950', {'rela': 'mapped_to', 'dir': 'N', 'rui': 'R148171650'}),
 ('C0000039-C0381030', {'rela': 'mapped_to', 'dir': 'N', 'rui': 'R148166148'}),
 ('C0000039-C0615231', {'rela': 'mapped_to', 'dir': 'N', 'rui': 'R148263106'})]

In [40]:
# notes.head()

## Examine semantic relations

### Definition of semantic types

https://uts.nlm.nih.gov/uts/umls/semantic-network/T187

In [22]:
stypes = pd.read_csv('data/semantic_types_definitions.csv')

In [23]:
stypes.head()

,RT,UI,STY_RL,STN_RTN,DEF,EX,UN,NH,ABR,RIN
0,STY,T001,Organism,A1.1,"Generally, a living individual, including all ...",NaN,NaN,NaN,orgm,NaN
1,STY,T002,Plant,A1.1.3.3,"An organism having cellulose cell walls, growi...",NaN,NaN,NaN,plnt,NaN
2,STY,T004,Fungus,A1.1.3.2,A eukaryotic organism characterized by the abs...,NaN,NaN,NaN,fngs,NaN
3,STY,T005,Virus,A1.1.4,An organism consisting of a core of a single n...,NaN,NaN,NaN,virs,NaN
4,STY,T007,Bacterium,A1.1.2,"A small, typically one-celled, prokaryotic mic...",NaN,NaN,NaN,bact,NaN


### Relations used in SemRep -- NOT USEFUL

In [24]:
data = []
with open('data/semRep-ontology.txt') as fo:
    for line in fo.readlines():
        if line.startswith('UMLS'):
            data.append(line.strip().split('|')[-1].split('-'))
ont = pd.DataFrame(data, columns=['STY1', 'RL', 'STY2'])

In [25]:
abr2tui = name2tui = pd.Series(stypes.UI.values,index=stypes.ABR).to_dict()
ont['STY1_TUI'] = ont['STY1'].map(abr2tui)
ont['STY2_TUI'] = ont['STY2'].map(abr2tui)

In [26]:
ont.sample(10)

,STY1,RL,STY2,STY1_TUI,STY2_TUI
466,neop,affects,neop,T191,T191
3013,drdd,treats,neop,T203,T191
2786,patf,process_of,fish,T046,T013
1865,rcpt,interacts_with,chvs,T192,T104
1075,patf,coexists_with,emod,T046,T050
2771,orgf,process_of,vtbt,T040,T010
1785,irda,interacts_with,inch,T130,T197
2475,bpoc,part_of,euka,T023,T204
583,patf,affects,npop,T046,T070
2460,anst,part_of,fish,T017,T013


In [39]:
ont['pair'] = ont[['STY1_TUI', 'STY2_TUI']].apply(lambda x : '{}-{}'.format(x[0],x[1]) if str(x[0])<str(x[1]) else '{}-{}'.format(x[1],x[0]), axis=1)

In [27]:
# ont[ont['RL']=='treats']

### Read semantic networks --- NOT USEFUL

In [28]:
snet = pd.read_csv('data/semantic_networks.csv')

In [29]:
# set(snet['RL'])

In [30]:
name2tui = pd.Series(stypes.UI.values,index=stypes.STY_RL).to_dict()

In [31]:
snet['STY1_TUI'] = snet['STY1'].map(name2tui)
snet['STY2_TUI'] = snet['STY2'].map(name2tui)

In [32]:
# stypes

In [33]:
type2chapter = pd.Series(stypes.STN_RTN.values, index=stypes.UI).to_dict()

In [38]:
snet['pair'] = snet[['STY1_TUI', 'STY2_TUI']].apply(lambda x : '{}-{}'.format(x[0],x[1]) if str(x[0])<str(x[1]) else '{}-{}'.format(x[1],x[0]), axis=1)

In [40]:
# set(snet[snet['STY1']=='Disease or Syndrome']['RL'])

In [41]:
# set(snet[snet['STY2']=='Disease or Syndrome']['RL'])

In [42]:
# 'treats'

In [43]:
rel_dup = snet.groupby(['pair'])['RL'].agg(nuni='nunique', uni='unique').reset_index()

In [44]:
rel_dup['rels'] = rel_dup.uni.apply(lambda x: ', '.join((x)))

In [45]:
rel_dup[rel_dup['rels']=='treats']

,pair,nuni,uni,rels
2305,T061-T184,1,[treats],treats
2592,T074-T184,1,[treats],treats
3003,T121-T184,1,[treats],treats
3239,T184-T195,1,[treats],treats
3241,T184-T203,1,[treats],treats


In [46]:
snet[snet['pair']=='T074-T184']

,STY1,RL,STY2,STY1_TUI,STY2_TUI,pair
3750,Medical Device,treats,Sign or Symptom,T074,T184,T074-T184


In [47]:
# rel_dup[rel_dup['uni']==['treats']]

In [48]:
# set(i[0] for i in (rel_dup[rel_dup['nuni']==1]['uni']))

In [49]:
# snet[snet['pair']=='T047-T050']

## NOTES:

Relations are not totally invertable and a default "associated_with" relation is assigned if the relations are not clear.
![disease-sign](img/diseas-sign.png)

![sign_disease](img/sign_disease.png)

![treats-diseases](img/treats-diseases.png)

A single relation can be related to multiple different pairs of semtypes, and the same pair of semtypes can have multiple different relations. such as

![anti-abnormal](img/anti-abnormal.png)




The structure of semantic networks: https://uts.nlm.nih.gov/uts/umls/semantic-network

(H) "isa" and (R) "associated_with" are two main categories. That is why every relation isa "associated_with". 

When the relations can be duplicated, how to assign a relation to an pair of concepts. 

Some pairs have specific relations, while some have more general relations, even if they are from the same semantic types. such as "treats", "prevents", "affects". 

a) If keep all relations, need to priorize the assignment of relations, from specific to general. Howeve, there is clear 

b) If remove all general relations, the assumption will be that all relations should be specfic, which will not hold in all cases. 

The choice of ralations is also dependent on the concept extraction tool, how general or specific the tool can extract. 

"REL" in "MRREL" table is not related to the information in Semantic Networks. "RIN" in "SRDEF" is Inverse of the Relation (RL) https://www.ncbi.nlm.nih.gov/books/NBK9679/table/ch05.T.srdef/

Abbrevations of UMLS: https://www.nlm.nih.gov/research/umls/knowledge_sources/metathesaurus/release/abbreviations.html#mrdoc_RELA

In [121]:
# snet = snet[snet['RL']!='isa']

In [122]:
# set(snet[snet['pair'].isin(rel_overlap[rel_overlap['nuni']==1]['pair'])]['RL'])

In [94]:
# set(snet['RL'])

In [16]:
# rset = stypes[stypes['STY_RL'].str.lower().isin(set(snet['RL']))]

In [17]:
# rset

### The same pair of types can have different relations, how to choose a relevant one

In [50]:
snet.head()

,STY1,RL,STY2,STY1_TUI,STY2_TUI,pair
0,Acquired Abnormality,affects,Amphibian,T020,T011,T011-T020
1,Acquired Abnormality,affects,Animal,T020,T008,T008-T020
2,Acquired Abnormality,affects,Archaeon,T020,T194,T020-T194
3,Acquired Abnormality,affects,Bacterium,T020,T007,T007-T020
4,Acquired Abnormality,affects,Bird,T020,T012,T012-T020


### Some documents are full of metrics and not provide useful language. So focus on discharge summary

In [91]:
# notes = pd.read_csv('/users/k1810895/data/KER/data/mimic_notes.csv')

In [92]:
# discharg_dids = set(notes[notes['category']=='Discharge summary']['row_id'].tolist())

In [93]:
# del notes

In [94]:
# len(discharg_dids)

In [5]:
discharg_dids = pd.read_csv('data/mimic_discharge_note_ids.csv')

In [6]:
discharg_dids = set(discharg_dids['row_id'].tolist())

In [7]:
len(discharg_dids)

59652

### Compare Medcat and Scispacy annotation granularity 

In [4]:
apath = '/users/k1810895/data/KER/data/scispacy_sentences/'
bpath = '/users/k1810895/data/KER/data/medcat_sentence/'

In [100]:
def sentence_annots(path):
    annot_counts = []
    annot_types = []
    annot_chapters = []
    str_len = []
    for index in range(1000):
        file_name = 'batch' + str(index) + '.json'

        docs = json.load(open(path+file_name, 'r'))

        for did in docs.keys():
            if int(did) in discharg_dids:
#             if True:
                sents = docs[did]
                for sid in sents.keys():
                    sent = sents[sid]
                    text = sent['sent_text']
                    annots = sent['sent_annots']
                    annot_counts.append(len(annots))
                    for annot in annots:
#                       Choose the first semantic type if multiple types are available
                        type_str = annot['types'].split(',')[0]
#                         annot_types.append(type_str)
                        str_len.append(len(annot['text']))
                        cha = len(type2chapter[type_str].split('.'))
                        annot_chapters.append(cha)
    return annot_counts, annot_chapters, str_len

In [101]:
def sentence_annots_medcat(path):
    annot_counts = []
    annot_types = []
    annot_chapters = []
    str_len = []
    for index in range(1000):
        file_name = 'batch' + str(index) + '.json'

        docs = json.load(open(path+file_name, 'r'))

        for did in docs.keys():
            if int(did) in discharg_dids:
#             if True:
                sents = docs[did]
                for sid in sents.keys():
                    sent = sents[sid]
                    text = sent['sent_text']
                    annots = sent['sent_annots']
                    annot_counts.append(len(annots))
                    for annot in annots:
#                       Choose the first semantic type if multiple types are available
                        str_len.append(len(annot['text']))
                        type_str = annot['tui'].split(',')[0]
                        cha = len(type2chapter[type_str].split('.'))
                        annot_chapters.append(cha)
    return annot_counts, annot_chapters, str_len

In [102]:
a_counts, a_chapters, a_str_lens = sentence_annots(apath)

In [ ]:
b_counts, b_chapters, b_str_lens = sentence_annots_medcat(bpath)

In [51]:
# pickle.dump((a_counts, a_chapters, a_str_lens), open('data/scispacy_annots_stat.pick', 'wb'))
# pickle.dump((b_counts, b_chapters, b_str_lens), open('data/medcat_annots_stat.pick', 'wb'))

In [3]:
a_counts, a_chapters, a_str_lens = pickle.load(open('data/scispacy_annots_stat.pick', 'rb'))
b_counts, b_chapters, b_str_lens = pickle.load(open('data/medcat_annots_stat.pick', 'rb'))

#### Distribution of annotations' chapters

In [9]:
len(a_counts)

5322252

In [12]:
# sns.histplot(a_chapters, label='Scispacy', color="skyblue")
# sns.histplot(b_chapters, label='MedCAT', color="red")
# plt.xlabel("Chapter level of entities")
# plt.ylabel("PDF (entities)")
# plt.legend()
# # plt.title("Scispacy on MIMIC")
# plt.show()

In [14]:
c = Counter(a_chapters)
c2 = Counter(b_chapters)
for i in range(1, 8):
    print('Scispacy', i, c[i], c[i]/len(a_chapters))
    print('MedCAT', i, c2[i], c2[i]/len(b_chapters))

Scispacy 1 272052 0.01225898574119883
MedCAT 1 662919 0.021306254178609107
Scispacy 2 3501830 0.15779661255238814
MedCAT 2 5986285 0.1923995387001957
Scispacy 3 9058066 0.4081671957450705
MedCAT 3 14643883 0.47065522840620483
Scispacy 4 4090146 0.18430682918494048
MedCAT 4 4469441 0.1436480866927888
Scispacy 5 4481730 0.20195206859682352
MedCAT 5 4362863 0.14022266374089304
Scispacy 6 784014 0.03532860058702108
MedCAT 6 988393 0.03176700695915789
Scispacy 7 4210 0.00018970759255747824
MedCAT 7 38 1.2213221506506015e-06


#### Scispacy produces more fine grained concepts than Medcat

In [15]:
# sns.distplot(a_counts)
# plt.xlabel("Number of entities in a single sentence")
# plt.ylabel("PDF (sentences)")
# plt.title("Scispacy on MIMIC")
# plt.show()

In [16]:
c = Counter(a_counts)
c2 = Counter(b_counts)
for i in range(1, 8):
    print('Scispacy', i, c[i], c[i]/len(a_counts))
    print('MedCAT', i, c2[i], c2[i]/len(b_counts))

Scispacy 1 1017556 0.19118899293006042
MedCAT 1 2493600 0.28484658653585837
Scispacy 2 1071471 0.20131910326681263
MedCAT 2 1771240 0.2023306335963161
Scispacy 3 934299 0.17554580279175055
MedCAT 3 1282471 0.14649802962834008
Scispacy 4 721402 0.1355445025902569
MedCAT 4 943554 0.10778317938412549
Scispacy 5 504616 0.0948124966649456
MedCAT 5 682485 0.07796098917706341
Scispacy 6 343141 0.06447289605978823
MedCAT 6 473730 0.05411468296424134
Scispacy 7 207860 0.039054896310809786
MedCAT 7 327024 0.03735630017456792


In [18]:
# sns.distplot(b_counts)
# plt.xlabel("Number of entities in a single sentence")
# plt.ylabel("PDF (sentences)")
# plt.title("Medcat on MIMIC")
# plt.show()

In [20]:
print(stats.describe(a_counts))
print(stats.describe(b_counts))

DescribeResult(nobs=5322252, minmax=(1, 315), mean=4.169672537113988, variance=23.177535766743144, skewness=7.137712520076788, kurtosis=101.30446871524101)
DescribeResult(nobs=8754186, minmax=(1, 160), mean=3.5541650588644105, variance=11.60929167019811, skewness=4.334075299061011, kurtosis=44.70064103162898)


#### Scispacy produces a larger number of annotations than Medcat. A possible reason is that Medcat use the longest string match in choosing concept candidates

In [21]:
print(stats.describe(a_str_lens))
print(stats.describe(b_str_lens))

DescribeResult(nobs=22192048, minmax=(1, 184), mean=8.970998350400107, variance=32.20667261403639, skewness=1.676493374928002, kurtosis=4.770225798523288)
DescribeResult(nobs=31113822, minmax=(3, 70), mean=9.092902761994331, variance=26.84527133431438, skewness=1.6642091664564336, kurtosis=3.9099507773475866)


string length in Medcat annotations is longer than scispacy

Some sentences contain too many concepts. Following Mintz's paper, only focus on setences contaning two entities in training data but consider all pairs of entities in test data. The training data will tend to be short sentences. However, the concepts are too details and the RELA are too specific; Use semantic types and semantic relations instead.

### Scispacy does not allow to choose a subset of voc. 

In [88]:
def screen_sentences(path, output_file):
    with open(output_file, 'w', buffering=128) as fw:
        for index in range(1000):

# path = apath
# index = 0
            file_name = 'batch' + str(index) + '.json'

            docs = json.load(open(path+file_name, 'r'))

            for did in docs.keys():
            #   Only for discharge summary since the quality of the content is relatively higher
            #   if int(did) in discharg_dids:
                if True:
                    sents = docs[did]
                    for sid in sents.keys():
                        sent = sents[sid]
                        text = sent['sent_text']
                        doc = nlp(text)
                        sentence_tokens = []
                        for s in doc.sents:
                            for token in s:
                                sentence_tokens.append(token.text)


                        annots = sent['sent_annots']
                        entities = []
                        for i, ann in enumerate(annots):
                            ann['id'] = i
                            span = doc.char_span(ann['start_char'], ann['end_char'])
                            ann['start'] = span.start
                            ann['end'] = span.end
                            entities.append(ann)

                        relations = []
                        # If include all sentence with any number of annotations. it ends with a super large file
                        # Select centain types of annotations to focus. 
                        for index_ai, ai in enumerate(entities):
                            for index_aj, aj in enumerate(entities[index_ai+1:]):
                                index_aj = index_aj + index_ai + 1
                                ai_cui, aj_cui = ai['cui'], aj['cui']

                                min_cui, max_cui = ai_cui, aj_cui
                                min_cui_index, max_cui_index = index_ai, index_aj

                                if min_cui > max_cui:
                                    min_cui, max_cui = aj_cui, ai_cui
                                    min_cui_index, max_cui_index = index_aj, index_ai

                                cui_key = min_cui + '-' + max_cui                    
                                if cui_key in rels:

                                    rel = rels[cui_key]['rela']
                                    dire = rels[cui_key]['dir']
                                    if dire == 'Y':
                                        head, tail = min_cui_index, max_cui_index

                                    else:
                                        head, tail = max_cui_index, min_cui_index
            #                         print(cui_key, rels[cui_key])
            #                         print({'type': rel,  'head': (entities[head]['cui']), 'tail': entities[tail]['cui']})
                                    relations.append({'type': rel,  'head': head, 'tail': tail})
                    fw.write(json.dumps({'text': text, 'tokens': sentence_tokens, 'entities': entities, 'relations': relations}))
                    fw.write('\n')


In [92]:
# screen_sentences(apath, '/users/k1810895/data/KER/data/scispacy_sentences.txt')

In [93]:
# screen_sentences(bpath, '/users/k1810895/data/KER/data/medcat_sentences.txt')

#### There are 59345 relevant positive sentences from scispacy annotations. and 129748 relevant positive sentences from medcat annotations.  There are few relations in a single sentence. SO USE the whole documents

The formatt {"text": "Directed by Andrei Zagdansky Not rated ; 72 minutes In the documentary '' Orange Winter '' orange blooms throughout Kiev , Ukraine , the epicenter of dissent over that country 's stolen 2004 presidential elections .", "relation": "/location/country/capital", "h": {"id": "m.07t21", "name": "Ukraine", "pos": [123, 130]}, "t": {"id": "m.02sn34", "name": "Kiev", "pos": [116, 120]}}

## Negation relations

Rule-based method: https://github.com/lhncbc/SemRep 


In [ ]:


A treats B 
^        ^ 
|        | 
c        d